In [ ]:
import time
import numpy as np
from math import sin, cos, atan2
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

MAC = '''
ROBOT_NAME = "Pioneer_p3dx" # MAC
LEFT_MOTOR_NAME = ROBOT_NAME + "_leftMotor" # MAC
RIGHT_MOTOR_NAME = ROBOT_NAME + "_rightMotor" # MAC
SENSOR_PATTERN = "Pioneer_p3dx_ultrasonicSensor{}" # MAC
'''
#LINUX = '''
ROBOT_NAME = "PioneerP3DX" # Linux
LEFT_MOTOR_NAME = "/PioneerP3DX/leftMotor" # Linux
RIGHT_MOTOR_NAME = "/PioneerP3DX/rightMotor" #L inux
SENSOR_PATTERN = "/PioneerP3DX/ultrasonicSensor[{}]" # Linux
#'''

L = 0.381      # distância entre rodas 
r = 0.0975     # raio da roda

# parametros para calibrar
K_att = 1.0
K_rep = 0.8
kv = 1.5
kw = 3.5

# limites e parâmetros do comportamento
d0 = 1.0 # raio da repulsão
v_max = 0.6
w_max = 1.5

# condição de parada
DIST_GOAL_STOP = 0.3 

def angle_diff(a, b):
    """Retorna diferença angular (a - b) normalizada para [-pi, pi]."""
    d = a - b
    return np.arctan2(np.sin(d), np.cos(d))

def clip(x, a, b):
    return max(a, min(b, x))

def transform_point_matrix(sim, mat4, point3):

    M = np.array(mat4).reshape((3,4))
    p = np.array([point3[0], point3[1], point3[2], 1.0])
    res = M @ p
    return res  # 3-vector

try:
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)
    
    # Frame que representa o Goal
    goal = 'Goal'
    goalFrame = sim.getObject('/' + goal)
    # Posição do Goal (x, y, theta)
    qgoal = sim.getObjectPosition(goalFrame, sim.handle_world)
    qgoal_orientation = sim.getObjectOrientation(goalFrame, sim.handle_world)



    MAC = '''
    robotHandle = sim.getObject('Pioneer_p3dx') # MAC
    l_wheel = sim.getObject('Pioneer_p3dx_leftMotor') # MAC
    r_wheel = sim.getObject('Pioneer_p3dx_rightMotor') # MAC
    sensorHandles = [sim.getObject(f'Pioneer_p3dx_ultrasonicSensor{i}') for i in range(1, 17)] # MAC
    '''
    #LINUX = '''
    robotHandle = sim.getObject('/PioneerP3DX') # Linux
    l_wheel = sim.getObject('/PioneerP3DX/leftMotor') # Linux
    r_wheel = sim.getObject('/PioneerP3DX/rightMotor') # Linux
    sensorHandles = [sim.getObject(f'/PioneerP3DX/ultrasonicSensor[{i}]') for i in range(16)] # Linux
    #'''

    # Configuração inicial (x, y, theta)
    q_xy = sim.getObjectPosition(robotHandle, sim.handle_world)
    q_orientation = sim.getObjectOrientation(robotHandle, sim.handle_world)
    q = np.array([q_xy[0], q_xy[1], q_orientation[2]])
    
    if len(sensorHandles) == 0:
        raise Exception("AVISO: nenhum sensor encontrado. Verifique nomes na cena.")
    
    # ------------------------------------------Forca de repusao no frame do robo ----------------------------------------------------------------------
    def compute_repulsion_force_robot_frame(sensor_handles, k_rep=K_rep, d0_local=d0):
        F_rep = np.zeros(2)
        for sensorHandle in sensor_handles:
            res = sim.readProximitySensor(sensorHandle)
            detected, detectedPoint, distance = res[0], res[1], res[2]
            sensor_mat = sim.getObjectMatrix(sensorHandle)  
            robot_mat = sim.getObjectMatrix(robotHandle)
            robot_mat_inv = sim.getMatrixInverse(robot_mat)
            rel = sim.multiplyMatrices(robot_mat_inv, sensor_mat)  # 3x4 matrix
            obs_in_robot_3 = sim.multiplyVector(rel, detectedPoint)
            obs_in_robot = np.array(obs_in_robot_3)

            if not detected:
                continue
            if distance <= 0 or np.isnan(distance) or np.isinf(distance):
                continue
            if distance <= d0_local:
                dir_vec = -obs_in_robot[:2]
                norm_dir = np.linalg.norm(dir_vec)
                if norm_dir == 0:
                    continue
                dir_vec = dir_vec / norm_dir
                mag = k_rep * (1.0 / distance - 1.0 / d0_local) / (distance**2 + 1e-6)
                F_rep += mag * dir_vec
        return F_rep

    def compute_goal_attraction_force_robot_frame(k_att=K_att):
        robot_pos_world = sim.getObjectPosition(robotHandle, sim.handle_world)
        robot_orientation = sim.getObjectOrientation(robotHandle, sim.handle_world)
        robot_theta = robot_orientation[2]
        rel_goal_world = np.array([qgoal[0], qgoal[1]]) - np.array(robot_pos_world[0], robot_pos_world[1])
        # world -> robot rotation
        c, s = cos(robot_theta), sin(robot_theta)
        R_inv = np.array([[c, s], [-s, c]])
        rel_goal_robot = R_inv @ rel_goal_world
        F_att_robot = k_att * rel_goal_robot
        return F_att_robot, robot_pos_world, robot_theta

    #--------------------------------------------Resultante das forcas ------------------------------------------------------------------------------
    def control_from_force_robot_frame(F_total, robot_theta, k_v=kv, k_w=kw, vlim=v_max, wlim=w_max):
        Fx, Fy = F_total[0], F_total[1]
        v = k_v * Fx
        w = k_w * Fy
        v = float(np.clip(v, -vlim, vlim))
        w = float(np.clip(w, -wlim, wlim))
        return v, w

# ---------------------------Inicialização simulação

    
    # Lembrar de habilitar o 'Real-time mode'    
    # Parar a simulação se estiver executando
    initial_sim_state = sim.getSimulationState()
    if initial_sim_state != 0:
        sim.stopSimulation()
        time.sleep(1)
    # Inicia a simulação
    sim.startSimulation()
    sim.step()

    last_sim_time = sim.getSimulationTime()
    while True:
        sim_time = sim.getSimulationTime()
        dt = sim_time - last_sim_time if sim_time - last_sim_time > 0 else 1e-3
        last_sim_time = sim_time

        # ---- calcula forças no frame do robo ----
        F_att_robot, rob_pos_world, robot_theta = compute_goal_attraction_force_robot_frame()
        F_rep_robot = compute_repulsion_force_robot_frame(sensorHandles)
        F_total_robot = F_att_robot + F_rep_robot

        # evita estouros -> mag 1
        normF = np.linalg.norm(F_total_robot)
        if normF > 1e-6:
            F_total_robot = F_total_robot / normF

        # controlar (v,w) a partir de F_total em frame do robo 
        v, w = control_from_force_robot_frame(F_total_robot, robot_theta)

        # converte para velocidades das rodas 
        wr = (2.0 * v + w * L) / (2.0 * r)
        wl = (2.0 * v - w * L) / (2.0 * r)


        sim.setJointTargetVelocity(l_wheel, wl)
        sim.setJointTargetVelocity(r_wheel, wr)

        #  mesma matriz  de antes (Mdir @ [wr, wl])
        Mdir = np.array([
            [r * np.cos(q[2]) / 2.0, r * np.cos(q[2]) / 2.0],
            [r * np.sin(q[2]) / 2.0, r * np.sin(q[2]) / 2.0],
            [r / L, -r / L]
        ])
        u = np.array([wr, wl])
        q = q + (Mdir @ u) * dt

        gt_pos = sim.getObjectPosition(robotHandle, sim.handle_world)
        gt_ori = sim.getObjectOrientation(robotHandle, sim.handle_world)
        gt_xy = np.array(gt_pos[:2])
        gt_theta = gt_ori[2]

        dist_goal = np.linalg.norm(np.array(qgoal[0], qgoal[1]) - gt_xy)

        print(f"[t={sim_time:.2f}] GT_pos={gt_xy.round(3)} θ={np.degrees(gt_theta):.1f}° | odom={q[:2].round(3)} θ={np.degrees(q[2]):.1f}° | dist_goal={dist_goal:.2f} | v={v:.3f} w={w:.3f}")

        if dist_goal <= DIST_GOAL_STOP:
            print("Chegou perto do objetivo — parando.")
            break

        sim.step()


    sim.setJointTargetVelocity(l_wheel, 0.0)
    sim.setJointTargetVelocity(r_wheel, 0.0)
    time.sleep(0.2)
    sim.stopSimulation()
    print("Simulação parada.")


except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: Cannot interpret '0.3251012499445407' as a data type
